## Project Data Download-Tiingo Data Download
### Context
Machine learning is widely used in financial trading strategy development. Technical data is the most common financial data to development forecasting model in financial industry.
### Content
In this section, I download the technical data for all stocks from NYSE and NASDAQ which are available on Tiingo by using the Tiingo API.
The relevant website is:https://www.tiingo.com/

#### 1. Module initialization

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime as DT
from datetime import timedelta
import os
from time import sleep
from itertools import chain

#### 2. Initial Setting

In [2]:
os.chdir(r'C:\UCSD_ML_Capstone\01_Input')
StockAll = pd.read_excel('./01_01_DataCodes/AllTickers_20211018.xlsx')
StockTickers  = list(chain(*StockAll.values))
Token2 = 'ddff0c680755cd0a68288e345d4e422e30e2c07b'
TokenList = [Token2]

## Request daily data
token_num = 0
Token = TokenList[token_num]
TotalTime = 0
ErrorTicker = []
headers = { 'Content-Type': 'application/json' }

#### 3. Request Setting

In [3]:
S_time = DT.now()
start_iTicker = 0
i_token = 0
## Avoid data request overwhelming, add some pause points
wait_itoken = 1400
sleepseconds = 60*10

#### 4. Data Request

In [4]:
for iTicker in range(start_iTicker,len(StockTickers)):
    i_token += 1
    if i_token > wait_itoken:
        print("Out of token at " + str(token_num))
        # reset counters
        i_token = 0
        print('Awake at '+ str(DT.now() + timedelta(seconds=sleepseconds)) )
        sleep(sleepseconds)
    
    Ticker = StockTickers[iTicker]
    Loop_start = DT.now()   
    
    try:
        reqdata = requests.get("https://api.tiingo.com/tiingo/daily/"+Ticker+"/"+\
                        "prices?startDate=2000-01-01&"+\
                        # "resampleFreq=1day&"+\
                        "columns=open,high,low,close,volume,adjOpen,adjHigh,adjLow,adjClose,adjVolume,divCash,splitFactor&"+\
                        "token="+Token, headers=headers)
    
        NewData = pd.DataFrame(reqdata.json())
            
        sleepseconds2 = 0.1
        sleep(sleepseconds2)
    
        NewData.to_pickle("./01_02_RawData/Tiingo_Stock_daily/" + Ticker + "_daily.zip")
    
        # calculate time
        Loop_end = DT.now()
        LoopTime = Loop_end-Loop_start
        TotalTime = Loop_end-S_time
        AvgTime = (TotalTime- timedelta(seconds=sleepseconds * ((iTicker-1)//wait_itoken) ) )\
            /(iTicker-start_iTicker+1)
        EstEnd = DT.now() + (len(StockTickers) - iTicker - 1)*AvgTime +\
            timedelta(seconds=sleepseconds * ((len(StockTickers) - iTicker - 1)//wait_itoken) )
        print('Done No.'+ str(iTicker+1) + \
                " " +Ticker+\
              "; Loop time:" + str(LoopTime) +\
              "; Total time:" + str(TotalTime) + \
              "; Average time:" + str(AvgTime) +\
              "; Estimate end:" + str(EstEnd) + '\n'  )
        
    except ValueError:
        print('Done No.'+ str(iTicker+1) +' '+Ticker + ' is unavailable.')
        ErrorTicker.append(Ticker)
        continue   

Done No.1 A; Loop time:0:00:01.085099; Total time:0:00:01.118010; Average time:0:10:01.118010; Estimate end:2021-12-17 10:07:37.548459

Done No.2 AA; Loop time:0:00:00.584439; Total time:0:00:01.703446; Average time:0:00:00.851723; Estimate end:2021-11-12 01:45:02.413916

Done No.3 AAC; Loop time:0:00:00.396949; Total time:0:00:02.101393; Average time:0:00:00.700464; Estimate end:2021-11-12 01:32:12.656866

Done No.4 AACG; Loop time:0:00:00.708107; Total time:0:00:02.811494; Average time:0:00:00.702874; Estimate end:2021-11-12 01:32:24.921353

Done No.5 AAIC; Loop time:0:00:00.843741; Total time:0:00:03.656232; Average time:0:00:00.731246; Estimate end:2021-11-12 01:34:49.306465

Done No.6 AAL; Loop time:0:00:00.824796; Total time:0:00:04.482025; Average time:0:00:00.747004; Estimate end:2021-11-12 01:36:09.498926

Done No.7 AAMC; Loop time:0:00:00.645276; Total time:0:00:05.128298; Average time:0:00:00.732614; Estimate end:2021-11-12 01:34:56.268215

Done No.8 AAME; Loop time:0:00:00.

Done No.60 ACHV; Loop time:0:00:00.779915; Total time:0:00:39.353772; Average time:0:00:00.655896; Estimate end:2021-11-12 01:28:25.850325

Done No.61 ACI; Loop time:0:00:00.419877; Total time:0:00:39.774647; Average time:0:00:00.652043; Estimate end:2021-11-12 01:28:06.242420

Done No.62 ACIC; Loop time:0:00:00.458772; Total time:0:00:40.234418; Average time:0:00:00.648942; Estimate end:2021-11-12 01:27:50.461421

Done No.63 ACII; Loop time:0:00:00.359041; Total time:0:00:40.594456; Average time:0:00:00.644356; Estimate end:2021-11-12 01:27:27.123281

Done No.64 ACIU; Loop time:0:00:00.547536; Total time:0:00:41.143985; Average time:0:00:00.642875; Estimate end:2021-11-12 01:27:19.586429

Done No.65 ACIW; Loop time:0:00:00.844743; Total time:0:00:41.989724; Average time:0:00:00.645996; Estimate end:2021-11-12 01:27:35.469197

Done No.66 ACKIT; Loop time:0:00:00.384968; Total time:0:00:42.374692; Average time:0:00:00.642041; Estimate end:2021-11-12 01:27:15.342204

Done No.67 ACLS; Loo

Done No.119 AEE; Loop time:0:00:00.926522; Total time:0:01:17.029570; Average time:0:00:00.647307; Estimate end:2021-11-12 01:27:42.140929

Done No.120 AEG; Loop time:0:00:00.816815; Total time:0:01:17.847383; Average time:0:00:00.648728; Estimate end:2021-11-12 01:27:49.372384

Done No.121 AEHL; Loop time:0:00:00.764954; Total time:0:01:18.613334; Average time:0:00:00.649697; Estimate end:2021-11-12 01:27:54.303599

Done No.122 AEHR; Loop time:0:00:00.759969; Total time:0:01:19.375298; Average time:0:00:00.650617; Estimate end:2021-11-12 01:27:58.985506

Done No.123 AEI; Loop time:0:00:00.411899; Total time:0:01:19.788192; Average time:0:00:00.648684; Estimate end:2021-11-12 01:27:49.148505

Done No.124 AEIS; Loop time:0:00:00.951454; Total time:0:01:20.740644; Average time:0:00:00.651134; Estimate end:2021-11-12 01:28:01.616523

Done No.125 AEL; Loop time:0:00:00.869678; Total time:0:01:21.612321; Average time:0:00:00.652899; Estimate end:2021-11-12 01:28:10.598526

Done No.126 AEM; 

Done No.179 AHAC; Loop time:0:00:01.574787; Total time:0:01:59.226024; Average time:0:00:00.666067; Estimate end:2021-11-12 01:29:17.610563

Done No.180 AHCO; Loop time:0:00:00.502653; Total time:0:01:59.729675; Average time:0:00:00.665165; Estimate end:2021-11-12 01:29:13.020229

Done No.181 AHH; Loop time:0:00:00.668214; Total time:0:02:00.398888; Average time:0:00:00.665187; Estimate end:2021-11-12 01:29:13.132253

Done No.182 AHPI; Loop time:0:00:02.106367; Total time:0:02:02.506252; Average time:0:00:00.673111; Estimate end:2021-11-12 01:29:53.457498

Done No.183 AHT; Loop time:0:00:00.775922; Total time:0:02:03.283172; Average time:0:00:00.673679; Estimate end:2021-11-12 01:29:56.347915

Done No.184 AI; Loop time:0:00:01.725386; Total time:0:02:05.009556; Average time:0:00:00.679400; Estimate end:2021-11-12 01:30:25.462125

Done No.185 AIG; Loop time:0:00:00.959436; Total time:0:02:05.969989; Average time:0:00:00.680919; Estimate end:2021-11-12 01:30:33.192334

Done No.186 AIH; L

Done No.239 ALKS; Loop time:0:00:00.768942; Total time:0:02:45.737773; Average time:0:00:00.693463; Estimate end:2021-11-12 01:31:37.028892

Done No.240 ALKT; Loop time:0:00:00.368015; Total time:0:02:46.107783; Average time:0:00:00.692116; Estimate end:2021-11-12 01:31:30.173836

Done No.241 ALL; Loop time:0:00:01.129980; Total time:0:02:47.238760; Average time:0:00:00.693937; Estimate end:2021-11-12 01:31:39.440905

Done No.242 ALLE; Loop time:0:00:00.788892; Total time:0:02:48.029646; Average time:0:00:00.694337; Estimate end:2021-11-12 01:31:41.476654

Done No.243 ALLK; Loop time:0:00:00.508638; Total time:0:02:48.539280; Average time:0:00:00.693577; Estimate end:2021-11-12 01:31:37.608991

Done No.244 ALLO; Loop time:0:00:00.504650; Total time:0:02:49.044928; Average time:0:00:00.692807; Estimate end:2021-11-12 01:31:33.690412

Done No.245 ALLT; Loop time:0:00:00.829782; Total time:0:02:49.875707; Average time:0:00:00.693370; Estimate end:2021-11-12 01:31:36.555556

Done No.246 AL

...

Done No.4855 WBS; Loop time:0:00:00.774241; Total time:1:22:48.489600; Average time:0:00:00.652624; Estimate end:2021-11-12 01:28:09.199185

Done No.4856 WBT; Loop time:0:00:00.570732; Total time:1:22:49.061331; Average time:0:00:00.652607; Estimate end:2021-11-12 01:28:09.114331

Done No.4857 WCC; Loop time:0:00:00.838991; Total time:1:22:49.901292; Average time:0:00:00.652646; Estimate end:2021-11-12 01:28:09.310733

Done No.4858 WCN; Loop time:0:00:00.789888; Total time:1:22:50.694156; Average time:0:00:00.652675; Estimate end:2021-11-12 01:28:09.457650

Done No.4859 WD; Loop time:0:00:00.700139; Total time:1:22:51.396295; Average time:0:00:00.652685; Estimate end:2021-11-12 01:28:09.509414

Done No.4860 WDAY; Loop time:0:00:00.653246; Total time:1:22:52.050526; Average time:0:00:00.652685; Estimate end:2021-11-12 01:28:09.510960

Done No.4861 WDC; Loop time:0:00:00.868678; Total time:1:22:52.920202; Average time:0:00:00.652730; Estimate end:2021-11-12 01:28:09.738211

Done No.4862 

Done No.4913 WMK; Loop time:0:00:00.851207; Total time:1:23:26.748707; Average time:0:00:00.652707; Estimate end:2021-11-12 01:28:09.620708

Done No.4914 WMPN; Loop time:0:00:00.697660; Total time:1:23:27.447364; Average time:0:00:00.652716; Estimate end:2021-11-12 01:28:09.668233

Done No.4915 WMS; Loop time:0:00:00.564006; Total time:1:23:28.012362; Average time:0:00:00.652698; Estimate end:2021-11-12 01:28:09.577383

Done No.4916 WMT; Loop time:0:00:00.836417; Total time:1:23:28.849776; Average time:0:00:00.652736; Estimate end:2021-11-12 01:28:09.768673

Done No.4917 WNC; Loop time:0:00:00.843263; Total time:1:23:29.694034; Average time:0:00:00.652775; Estimate end:2021-11-12 01:28:09.966903

Done No.4918 WNEB; Loop time:0:00:00.863228; Total time:1:23:30.557262; Average time:0:00:00.652818; Estimate end:2021-11-12 01:28:10.184709

Done No.4919 WNS; Loop time:0:00:00.661232; Total time:1:23:31.219493; Average time:0:00:00.652820; Estimate end:2021-11-12 01:28:10.194462

Done No.492

Done No.4972 XBIT; Loop time:0:00:00.570155; Total time:1:24:06.895274; Average time:0:00:00.653036; Estimate end:2021-11-12 01:28:11.296055

Done No.4973 XCUR; Loop time:0:00:00.446253; Total time:1:24:07.342524; Average time:0:00:00.652995; Estimate end:2021-11-12 01:28:11.085513

Done No.4974 XEC; Loop time:0:00:00.913096; Total time:1:24:08.257622; Average time:0:00:00.653047; Estimate end:2021-11-12 01:28:11.353596

Done No.4975 XEL; Loop time:0:00:00.810350; Total time:1:24:09.068935; Average time:0:00:00.653079; Estimate end:2021-11-12 01:28:11.515510

Done No.4976 XELA; Loop time:0:00:00.598236; Total time:1:24:09.669169; Average time:0:00:00.653069; Estimate end:2021-11-12 01:28:11.461535

Done No.4977 XELB; Loop time:0:00:00.744060; Total time:1:24:10.415236; Average time:0:00:00.653087; Estimate end:2021-11-12 01:28:11.556549

Done No.4978 XENE; Loop time:0:00:00.504921; Total time:1:24:10.921161; Average time:0:00:00.653058; Estimate end:2021-11-12 01:28:11.406168

Done No.

Done No.5030 YOU; Loop time:0:00:00.500768; Total time:1:24:40.491180; Average time:0:00:00.652185; Estimate end:2021-11-12 01:28:06.965664

Done No.5031 YPF; Loop time:0:00:00.823510; Total time:1:24:41.315686; Average time:0:00:00.652219; Estimate end:2021-11-12 01:28:07.139957

Done No.5032 YQ; Loop time:0:00:00.424517; Total time:1:24:41.741220; Average time:0:00:00.652174; Estimate end:2021-11-12 01:28:06.910707

Done No.5033 YRD; Loop time:0:00:00.549721; Total time:1:24:42.292931; Average time:0:00:00.652154; Estimate end:2021-11-12 01:28:06.809124

Done No.5034 YSAC; Loop time:0:00:00.428224; Total time:1:24:42.722122; Average time:0:00:00.652110; Estimate end:2021-11-12 01:28:06.583741

Done No.5035 YSG; Loop time:0:00:00.432155; Total time:1:24:43.154277; Average time:0:00:00.652066; Estimate end:2021-11-12 01:28:06.361410

Done No.5036 YTEN; Loop time:0:00:00.721654; Total time:1:24:43.876929; Average time:0:00:00.652080; Estimate end:2021-11-12 01:28:06.432738

Done No.5037

Done No.5088 ZYNE; Loop time:0:00:00.561392; Total time:1:25:11.566998; Average time:0:00:00.650858; Estimate end:2021-11-12 01:28:00.213425

Done No.5089 ZYXI; Loop time:0:00:00.733569; Total time:1:25:12.301538; Average time:0:00:00.650875; Estimate end:2021-11-12 01:28:00.297107



#### 5.Outputs

In [5]:
## Unavailable ticker list
pd.Series(ErrorTicker).to_excel("./01_02_RawData/Tiingo_Stock_daily/" + "ErrorTicker_daily.xlsx", index=False)
## ID of the last run ticker
pd.Series(iTicker).to_excel("./01_02_RawData/Tiingo_Stock_daily/" + "iTicker_daily.xlsx", index=False)

In [ ]:
from os import listdir

AvaTickers = listdir("./01_02_RawData/Tiingo_Stock_daily" )
AvaTickers = pd.Series(list(set([x.split('_')[0] for x in AvaTickers])))
AvaTickers.name = 'Tickers'
AvaTickers.to_csv(r'C:\UCSD_ML_Capstone\01_Input\01_01_DataCodes\Tiingo_Available_Tickers.csv', index=False)